<a href="https://colab.research.google.com/github/maxwellowusu/movie_recommender_system_ML/blob/main/movie_recommender_max.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movies Recommendation System
## Machine Learning 1
Group Project by Maxwell, Michael, Yaswanth 




### Introduction 

### Motivation

### Research Question 
- Can we predict movie rating ? 

In [1]:

from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/Colab Notebooks/movie_recommender_project/'

# Get the absolute path of the shallow utilities folder
abspath_util_shallow = '/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/code/utilities/p2_shallow_learning/'

# Get the absolute path of the shallow models folder
abspath_model_shallow = '/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/code/models/p2_shallow_learning/'

Mounted at /content/drive


In [2]:
# Warning 

import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

In [3]:
# Matplotlib

import matplotlib.pyplot as plt
%matplotlib inline 

# Set matplotlib sizes
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=20)

In [4]:
# Tensorflow

# The magic below allows us to use tensorflow version 2.x
%tensorflow_version 2.x 
import tensorflow as tf
from tensorflow import keras

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [5]:
# The random seed
random_seed = 42

# Set random seed in tensorflow
tf.random.set_seed(random_seed)

# Set random seed in numpy
import numpy as np
np.random.seed(random_seed)

## Data preprocessing 

In [6]:
# Change working directory to the absolute path of the shallow utilities folder
%cd $abspath_util_shallow

# Import the shallow utitilities
%run pmlm_utilities_shallow.ipynb

/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/code/utilities/p2_shallow_learning


## Load data


In [7]:
# let load .dat data 
import pandas as pd

# Load data
movies=pd.read_csv(abspath_curr+'data/movies.dat', sep='::', encoding='latin-1', header=None, names=['MovieId', 'Title', 'Genres'])
ratings=pd.read_csv(abspath_curr+'data/ratings.dat', sep='::', encoding='latin-1', header=None, names=['UserId', 'MovieId', 'Rating', 'Timestamp'])
users=pd.read_csv(abspath_curr+'data/users.dat', sep='::', encoding='latin-1', header=None, names=['UserId', 'Gender', 'Age', 'Occupation', 'Zip-code'])

In [8]:
movies.head()

,MovieId,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
ratings.head()

,UserId,MovieId,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
users.head()

,UserId,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


Merge datasets
 - First merging movie and ratings data on 'MovieId'
 - Second merged movie+ratings data with users data on 'UserId'

In [14]:
# Merge data frames
data=pd.merge(left=movies, right=ratings, how='inner', on='MovieId')
data=pd.merge(left=data, right=users, how='inner', on='UserId')
print(data.shape)
data.head()

(1000209, 10)


,MovieId,Title,Genres,UserId,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067


In [15]:
from sklearn.model_selection import train_test_split

# Divide the training data into training (80%) and testing (20%)
df_train, df_test = train_test_split(data, train_size=0.80, random_state=random_seed)

# Divide the training data into training (80%) and validation (20%)
df_train, df_val = train_test_split(df_train, train_size=0.80, random_state=random_seed)

# Reset the index
df_train, df_val, df_test = df_train.reset_index(drop=True), df_val.reset_index(drop=True), df_test.reset_index(drop=True)

In [17]:
# print dimension
print("trainset shape:", df_train.shape)
print("validationset shape:", df_val.shape)
print("testset shape:", df_test.shape)

trainset shape: (640133, 10)
validationset shape: (160034, 10)
testset shape: (200042, 10)


## Handling data time variables 

In [20]:
# Get the date time variables
datetime_vars = []

# Call datetime_transformer on df_train
# See the implementation in pmlm_utilities.ipynb
df_train = datetime_transformer(df_train, datetime_vars)

# Print the first 5 rows of df_train
df_train.head()

# Call datetime_transformer on df_val
# See the implementation in pmlm_utilities.ipynb
df_val = datetime_transformer(df_val, datetime_vars)

# Print the first 5 rows of df_val
df_val.head()

# Call datetime_transformer on df_test
# See the implementation in pmlm_utilities.ipynb
df_test = datetime_transformer(df_test, datetime_vars)

# Print the first 5 rows of df_test
df_test.head()

,MovieId,Title,Genres,UserId,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,588,Aladdin (1992),Animation|Children's|Comedy|Musical,510,5,976207924,M,18,12,55109
1,3203,Dead Calm (1989),Thriller,2279,4,974561199,M,50,7,08816
2,586,Home Alone (1990),Children's|Comedy,678,4,989239492,M,25,0,34952
3,345,"Adventures of Priscilla, Queen of the Desert, ...",Comedy|Drama,733,5,975482948,M,35,16,30034
4,1041,Secrets & Lies (1996),Drama,4771,3,993357573,F,25,4,04101


## Handling missing data

In [21]:
# Combine df_train, df_val and df_test
df = pd.concat([df_train, df_val, df_test], sort=False)

In [22]:
# Call nan_checker on df
# See the implementation in pmlm_utilities.ipynb
df_nan = nan_checker(df)

# Print df_nan
df_nan

,var,proportion,dtype


In [23]:
# Print the unique data type of variables with NaN
pd.DataFrame(df_nan['dtype'].unique(), columns=['dtype'])

,dtype


In [24]:
# Get the variables with missing values, their proportion of missing values and data type
df_miss = df_nan[df_nan['dtype'] == 'float64'].reset_index(drop=True)

# Print df_miss
df_miss

,var,proportion,dtype


## Encoding data 

In [ ]:
# Combine df_train, df_val and df_test
df = pd.concat([df_train, df_val, df_test], sort=False)

# Print the unique data type of variables in df
pd.DataFrame(df.dtypes.unique(), columns=['dtype'])

### Identify categorical data

In [25]:
# Call cat_var_checker on df
# See the implementation in pmlm_utilities.ipynb
df_cat = cat_var_checker(df)

# Print the dataframe
df_cat

,var,nunique
0,Title,3706
1,Zip-code,3439
2,Genres,301
3,Gender,2


### Encode Gender
#### Gender (Binary Categorical Variable)
- Female = 0
- Male = 1

In [ ]:
# Encode gender inot 0 and 1
df['Gender']=np.where(df['Gender']=='F',0,1)

### Encode Genres
Genre (Multilabel Categorical Variable) using countvectorize 

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

def token_creator(genres):
  l = genres.split('|')
  return l

cv = CountVectorizer(analyzer=token_creator)

matrix_genres = cv.fit_transform(df.Genres)

df_genres=pd.DataFrame(matrix_genres.toarray(), columns=cv.get_feature_names())

In [27]:
df_genres.head()

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [ ]:
df_movies=pd.concat([data, df_genres], axis=1)
df_movies.drop(['Genres'], axis=1, inplace=True)